In [8]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [9]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Automotives, Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [10]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [11]:
test_df = pd.read_parquet('../test/')

In [12]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [13]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [14]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [15]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [16]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [21]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [32]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [42]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probabilty_y': 'merchant_fraud'})


In [43]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,fraud_probability_y
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0
